In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

/home/djulien/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [6]:
arch=resnet34
PATH = "../../data/dogbreeds/"
sz=224

In [7]:
os.listdir(PATH)

['train', 'labels.csv', '.ipynb_checkpoints', 'test']

In [8]:
files = os.listdir(f'{PATH}test')[:5]
files

['f1705303b30da3a48db2a5f34376b947.jpg',
 'a9b79809bc1301f8d148e15bfecf6485.jpg',
 'cc94a3bb7191a247f59e1540d8430c93.jpg',
 'f7b489a38302afc010d8a386a1b7d085.jpg',
 '6e21c96226078576342ca66e25eb1aa9.jpg']

In [21]:
data  =  ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', tfms=tfms_from_model(arch, sz), suffix='.jpg', test_name='test', val_idxs=get_cv_idxs(10222))

In [22]:
learn  =  ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 162/162 [00:59<00:00,  2.63it/s]


In [28]:
learn.fit(0.001, 10)

epoch      trn_loss   val_loss   accuracy   
    0      0.106099   0.556001   0.844423  
    1      0.101396   0.558788   0.844423  
    2      0.104018   0.559076   0.84638   
    3      0.112899   0.55886    0.844423  
    4      0.107195   0.554676   0.845401  
    5      0.105711   0.556669   0.847358  
    6      0.098947   0.556796   0.844912  
    7      0.111147   0.561504   0.847358  
    8      0.105806   0.560517   0.844423  
    9      0.102107   0.556721   0.84638   



[array([0.55672]), 0.8463796478661539]

In [31]:
log_preds, y = learn.predict_with_targs(is_test=True) # use test dataset rather than validation dataset
probs = np.exp(log_preds)

In [32]:
probs.shape

(10357, 120)

In [33]:
df=pd.DataFrame(probs)
df.columns=data.classes

In [35]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [36]:
df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,f1705303b30da3a48db2a5f34376b947,2.403968e-08,1.305394e-06,2.153659e-07,5.044334e-03,1.422208e-06,5.234767e-09,4.291563e-09,3.927096e-09,2.251603e-10,...,1.315987e-07,9.766988e-09,7.096321e-02,8.622334e-08,8.823345e-01,1.055724e-09,2.097464e-10,7.034335e-05,6.919243e-09,6.196209e-07
1,a9b79809bc1301f8d148e15bfecf6485,1.472832e-06,5.951112e-09,6.724267e-07,8.096612e-08,1.233882e-05,4.063248e-05,1.493946e-07,1.675789e-07,1.087841e-06,...,3.956308e-07,9.480037e-06,4.959707e-08,4.547112e-05,6.335005e-07,1.448883e-07,5.086103e-03,2.872194e-08,2.421804e-06,4.546511e-08
2,cc94a3bb7191a247f59e1540d8430c93,7.373529e-07,2.019878e-06,1.870293e-07,4.210785e-07,2.706899e-08,1.964585e-03,4.154167e-08,5.953831e-07,9.020866e-07,...,3.809958e-07,1.169502e-08,1.014059e-08,5.802526e-07,9.295457e-09,3.596765e-06,1.633958e-07,5.513422e-08,1.444288e-06,3.499455e-08
3,f7b489a38302afc010d8a386a1b7d085,2.223638e-08,1.445175e-07,1.057081e-08,3.365675e-08,3.081015e-08,5.883090e-04,7.644393e-10,4.298176e-08,2.021486e-07,...,8.001535e-08,1.670598e-11,5.742578e-10,2.749782e-07,2.787007e-09,2.716514e-06,7.716279e-09,1.138912e-07,1.299805e-08,6.234877e-10
4,6e21c96226078576342ca66e25eb1aa9,2.562899e-04,1.150846e-05,4.304947e-07,1.197078e-05,1.113106e-06,5.263109e-07,6.161175e-08,2.204639e-07,8.792846e-08,...,2.233461e-08,2.375298e-09,8.667836e-06,1.793426e-07,3.813821e-06,1.017577e-06,3.814022e-07,1.962067e-06,1.754131e-06,7.748250e-06


In [37]:
SUBM = f'../../out/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}dogbreeds_simple.gz', compression='gzip', index=False)